## Import library


In [2]:
import pandas as pd
import numpy as np
from collections import Counter

#Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Load Data
meload data dan konversi tag dan token menuju bentuk angka, untuk CRF

In [3]:
data = pd.read_csv("../Dataset/annotated_ner_data_new.csv")
data

,token,ner,tweet_id,index
0,Hidup,O,1382601382042103808,0
1,sesedih,O,1382601382042103808,1
2,dan,O,1382601382042103808,2
3,secaper,O,1382601382042103808,3
4,apa,O,1382601382042103808,4
...,...,...,...,...
170065,muntah,O,1386004972207153156,5
170066,Ngabuburit,O,1386301391086309382,0
170067,tadi,O,1386301391086309382,1
170068,sore,O,1386301391086309382,2


In [4]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['token'].to_list()))
    else:
        vocab = list(set(data['ner'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok


token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [5]:
data['token_idx'] = data['token'].map(token2idx)
data['ner_idx'] = data['ner'].map(tag2idx)
data.head()

,token,ner,tweet_id,index,token_idx,ner_idx
0,Hidup,O,1382601382042103808,0,3236,12
1,sesedih,O,1382601382042103808,1,10983,12
2,dan,O,1382601382042103808,2,27019,12
3,secaper,O,1382601382042103808,3,16994,12
4,apa,O,1382601382042103808,4,24743,12


In [6]:
# Fill na
data_fillna = data.fillna(method='ffill', axis=0)
# Groupby and collect columns
data_group = data_fillna.groupby(
['tweet_id'],as_index=False
)['token', 'ner', 'token_idx', 'ner_idx'].agg(lambda x: list(x))
# Visualise data
data_group.head()

ipykernel_launcher:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,tweet_id,token,ner,token_idx,ner_idx
0,1380681275594305537,"[@ridwankamil, @KickAndyShow, @Metro_TV, Upaya...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-P...","[157, 29367, 31047, 27638, 20471, 25292, 27146...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1..."
1,1380685209075544064,"[@GyuuPotter, iyhh, ,, soalnya, mwu, kerjain, ...","[O, O, O, O, O, O, O, O]","[11146, 10583, 8137, 14350, 15060, 23477, 1141...","[12, 12, 12, 12, 12, 12, 12, 12]"
2,1380685355897085952,"[@ShopeeID, Bismillah, yok, menang, ����, SHOP...","[O, O, O, O, O, B-PROD, O, O, O, O]","[2935, 9971, 9037, 9737, 11256, 9249, 2935, 23...","[12, 12, 12, 12, 12, 11, 12, 12, 12, 12]"
3,1380686115007393792,"[udah, 2, mingguan, tinggal, bareng, teteh, ,,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC,...","[35353, 10118, 12507, 23411, 3512, 16843, 8137...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1..."
4,1380688640611786754,"[@ShopeeID, Potong, buah, semangka, pakai, ala...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG,...","[2935, 23131, 1811, 409, 10287, 25562, 31463, ...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1..."


### contoh konversi

In [7]:
# data_group[data_group.tweet_id==1382601382042103808]
data_group[data_group.tweet_id==1380735296720019457]

,tweet_id,token,ner,token_idx,ner_idx
27,1380735296720019457,"[Gubernur, Khofifah, Terima, Lencana, Kehormat...","[O, B-PER, O, O, O, O, O, B-ORG, O]","[15344, 32605, 18219, 14832, 21327, 3053, 2676...","[12, 0, 12, 12, 12, 12, 12, 5, 12]"


In [9]:
# data[data.tweet_id==1382601382042103808]
data[data.tweet_id==1380735296720019457]

,token,ner,tweet_id,index,token_idx,ner_idx
60302,Gubernur,O,1380735296720019457,0,15344,12
60303,Khofifah,B-PER,1380735296720019457,1,32605,0
60304,Terima,O,1380735296720019457,2,18219,12
60305,Lencana,O,1380735296720019457,3,14832,12
60306,Kehormatan,O,1380735296720019457,4,21327,12
60307,Asthabrata,O,1380735296720019457,5,3053,12
60308,dari,O,1380735296720019457,6,26766,12
60309,IKAPTK,B-ORG,1380735296720019457,7,3908,5
60310,https://t.co/enN89BpcmE,O,1380735296720019457,8,12123,12


## memberi label one-hot encoding

In [16]:
# input: list of row -> [token, tag]
# output: list of all [full_entities,  NE] (O is excluded)
def get_entities(row):
    full_entities = []
    prev_tag = "O" #init default tag
    for i in range(len(row.token)):
        start_tag=row.ner[i][:1]
        tag= row.ner[i][2:] if row.ner[i] !='O' else 'O'
        if tag == "O":
            prev_tag = tag
            continue

        if start_tag == 'B': # Begin NE
            full_entities.append([row.token[i], tag])
        elif start_tag == 'I' and prev_tag == tag: # Inside NE
            full_entities[-1][0]  = full_entities[-1][0]+" "+ row.token[i]
        prev_tag = tag
    return full_entities


# input: list of fullentities with the tag
# output: mapping of one hot encoding
def one_hot_enc(ls):
    # print(list(ls[0].items()))
    dc={
        'PER':0,
        'LOC':0,
        'ORG':0,
        'PROD':0,
        'WA':0,
        'EV':0
    }

    for i,j in list(ls[0].items()):
        # dc[i]=j
        dc[i]=1
    # print(dc)
    return ''.join([str(i) for i in dc.values()])

In [17]:
data_group=data_group.assign(entities=[get_entities(i[1]) for i in data_group.iterrows()])
data_group=data_group.assign(count_entities=[[ Counter(j for _,j in i[1].entities)] for i in data_group.iterrows()])
data_group=data_group.assign(onehot_entities=[one_hot_enc(i[1].count_entities) for i in data_group.iterrows()])

In [18]:
data_group[data_group.tweet_id==1382601382042103808]

,tweet_id,token,ner,token_idx,ner_idx,entities,count_entities,onehot_entities
1376,1382601382042103808,"[Hidup, sesedih, dan, secaper, apa, yak, nyamp...","[O, O, O, O, O, O, O, B-PROD, O, B-PROD, O, O,...","[14774, 13688, 19108, 17427, 8317, 5731, 16172...","[9, 9, 9, 9, 9, 9, 9, 7, 9, 7, 9, 9, 9, 9, 9, ...","[[ig, PROD], [youtube, PROD], [ml, PROD]]",[{'PROD': 3}],000100


In [19]:
data_group.head()

,tweet_id,token,ner,token_idx,ner_idx,entities,count_entities,onehot_entities
0,1380681275594305537,"[@ridwankamil, @KickAndyShow, @Metro_TV, Upaya...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-P...","[7429, 19259, 28344, 18767, 13933, 18204, 1401...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 6, ...","[[gubernur, PER], [jabar, LOC]]","[{'PER': 1, 'LOC': 1}]",110000
1,1380685209075544064,"[@GyuuPotter, iyhh, ,, soalnya, mwu, kerjain, ...","[O, O, O, O, O, O, O, O]","[26285, 20550, 10560, 16466, 21996, 5742, 1252...","[9, 9, 9, 9, 9, 9, 9, 9]",[],[{}],000000
2,1380685355897085952,"[@ShopeeID, Bismillah, yok, menang, ����, SHOP...","[O, O, O, O, O, B-PROD, O, O, O, O]","[30694, 16325, 6049, 13298, 7045, 22088, 30694...","[9, 9, 9, 9, 9, 7, 9, 9, 9, 9]","[[SHOPEE, PROD]]",[{'PROD': 1}],000100
3,1380686115007393792,"[udah, 2, mingguan, tinggal, bareng, teteh, ,,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC,...","[706, 13336, 7454, 8032, 9392, 7487, 10560, 13...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 11, 9,...","[[jogja, LOC]]",[{'LOC': 1}],010000
4,1380688640611786754,"[@ShopeeID, Potong, buah, semangka, pakai, ala...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG,...","[30694, 10366, 26958, 29080, 23617, 24744, 198...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 1, 9, ...","[[MAMAMOO, ORG], [SHOPEE, PROD], [iPhone 12., ...","[{'ORG': 3, 'PROD': 4}]",001100


### hitung jumlah setiap OHE

In [20]:
Counter(data_group.onehot_entities)

Counter({'110000': 457,
         '000000': 2019,
         '000100': 808,
         '010000': 483,
         '001100': 81,
         '100000': 1377,
         '111100': 6,
         '001000': 167,
         '101000': 625,
         '101010': 14,
         '100100': 240,
         '110100': 54,
         '001101': 26,
         '101101': 8,
         '010010': 8,
         '000001': 99,
         '011000': 57,
         '111000': 178,
         '010100': 92,
         '100001': 172,
         '110001': 42,
         '100010': 28,
         '110010': 10,
         '101001': 66,
         '010001': 16,
         '001010': 5,
         '011001': 12,
         '001001': 45,
         '101100': 49,
         '000101': 60,
         '011110': 1,
         '000010': 26,
         '000110': 14,
         '001110': 4,
         '100110': 9,
         '111001': 18,
         '011100': 15,
         '001011': 3,
         '100101': 7,
         '101110': 2,
         '011010': 1,
         '000011': 3,
         '000111': 2,
         '11

## Melakukan Stratified K-Fold

In [21]:
X = data_group
y = data_group.onehot_entities
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=43)

for temp_index, test_index in skf.split(X, y):
    # print("TRAIN:", temp_index, "TEST:", test_index)
    X_temp, X_test = X.iloc[temp_index], X.iloc[test_index]
    y_temp, y_test = y[temp_index], y[test_index]
    # print(Counter(X.iloc[temp_index].onehot_entities).most_common(10),
    #     Counter(X.iloc[test_index].onehot_entities).most_common(10))
    # print(np.bincount(y[temp_index]), np.bincount(y[test_index]))
print()
for train_index, val_index in skf.split(X_temp, y_temp):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y[train_index], y[val_index]
    # print(Counter(X.iloc[train_index].onehot_entities).most_common(10),
    #     Counter(X.iloc[val_index].onehot_entities).most_common(10))
    # print(np.bincount(y[train_index]), np.bincount(y[test_index]))
# print(Counter(X.iloc[train_index].onehot_entities).most_common(10))
# print(Counter(X.iloc[test_index].onehot_entities).most_common(10))
train_l=X.iloc[train_index].onehot_entities.values
train_c=Counter(train_l)
test_l=X.iloc[test_index].onehot_entities.values
test_c=Counter(test_l)
val_l=X.iloc[val_index].onehot_entities.values
val_c=Counter(val_l)

print(sorted([(i, train_c[i] / len(train_l) * 100.0) for i in train_c],key=lambda x:x[1],reverse=True)[:5])
print(sorted([(i, test_c[i] / len(test_l) * 100.0) for i in test_c],key=lambda x:x[1],reverse=True)[:5])
print(sorted([(i, val_c[i] / len(val_l) * 100.0) for i in val_c],key=lambda x:x[1],reverse=True)[:5])



[('000000', 25.715488215488215), ('100000', 19.234006734006734), ('000100', 10.711279461279462), ('101000', 8.6489898989899), ('010000', 6.25)]
[('000000', 27.205387205387204), ('100000', 18.585858585858585), ('000100', 10.84175084175084), ('101000', 8.417508417508419), ('010000', 6.531986531986532)]
[('000000', 27.18855218855219), ('100000', 19.52861952861953), ('000100', 10.521885521885523), ('101000', 9.427609427609427), ('110000', 6.6498316498316505)]


d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [39]:
def flatten(t):
    return [item for sublist in t for item in sublist]

print(Counter(flatten(X.iloc[train_index].ner.values)))
print(Counter(flatten(X.iloc[test_index].ner.values)))
print(Counter(flatten(X.iloc[val_index].ner.values)))

# print(sorted([(i, train_c[i] / len(train_l) * 100.0) for i in train_c],key=lambda x:x[1],reverse=True)[:5])

Counter({'O': 95048, 'B-PER': 3694, 'B-PROD': 2270, 'I-PER': 2016, 'B-LOC': 1455, 'B-ORG': 1388, 'I-PROD': 792, 'I-ORG': 785, 'I-EV': 673, 'I-LOC': 615, 'B-EV': 524, 'I-WA': 159, 'B-WA': 99})
Counter({'O': 29507, 'B-PER': 1148, 'B-PROD': 676, 'I-PER': 603, 'B-LOC': 437, 'B-ORG': 389, 'I-PROD': 243, 'I-EV': 199, 'I-LOC': 196, 'I-ORG': 193, 'B-EV': 148, 'I-WA': 47, 'B-WA': 28})
Counter({'O': 23718, 'B-PER': 949, 'I-PER': 531, 'B-PROD': 465, 'B-ORG': 346, 'B-LOC': 336, 'I-EV': 163, 'I-PROD': 161, 'I-ORG': 160, 'I-LOC': 152, 'B-EV': 124, 'I-WA': 45, 'B-WA': 30})


## Export file 

In [ ]:
X.iloc[train_index].to_csv('../Dataset/StratifiedKFold/train.csv',index=False)
X.iloc[test_index].to_csv('../Dataset/StratifiedKFold/test.csv',index=False)
X.iloc[val_index].to_csv('../Dataset/StratifiedKFold/val.csv',index=False)
X.iloc[temp_index].to_csv('../Dataset/StratifiedKFold/trainval.csv',index=False)

## Explanatory Data Analysis

In [ ]:
def return_all_entities(data):
    full_entities = []
    prev_tag = "O"
    for index, row in data.iterrows():
        start_tag=row.ner[:1]
        tag= row.ner[2:] if row.ner !='O' else 'O'
        if tag == "O":
            prev_tag = tag
            continue

        if start_tag == 'B': # Begin NE
            full_entities.append([row.token,tag,row.tweet_id])
        elif start_tag == 'I' and prev_tag == tag: # Inside NE
            full_entities[-1][0]  = full_entities[-1][0]+" "+ row.token
        prev_tag = tag

    for row in full_entities:
        row.append(len(row[0].split(" ")))
        
    return pd.DataFrame(full_entities, columns=['token','ner','tweet_id','count_token'])

In [ ]:
all_entities=return_all_entities(data)
all_entities

In [ ]:
dict(all_entities.ner.value_counts())

In [ ]:
#count total of named entities within a tweet 
dict(all_entities.count_token.value_counts())

In [ ]:
#show all tweet that contains exactly 5 NEs 
all_entities[all_entities.count_token==5]

In [7]:
## count token length on all tweets
data.groupby('tweet_id').size().reset_index(name='token')

,tweet_id,token
0,1380681275594305537,29
1,1380685209075544064,8
2,1380685355897085952,10
3,1380686115007393792,21
4,1380688640611786754,41
...,...,...
7420,1391611459931541506,45
7421,1391612015286702084,34
7422,1391614165182128130,21
7423,1391614604388622337,53


In [ ]:
data.loc[data['ner'] == 'B-PER', 'token']

In [ ]:
data.groupby('ner').size().reset_index(name='count tag')
# data[data.tweet_id==1382900960721207300]

In [ ]:
#Words distribution across Tags without O tag
plt.figure(figsize=(10, 4))
ax = sns.countplot('ner', data=data.loc[data['ner'] != 'O'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
plt.tight_layout()
plt.show()